In [1]:
import tensorflow as tf
import numpy as np
import C3D_model
import time
import data_processing
import os
import os.path
from os.path import join
import pickle

#Run the code using selected GPU

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#TF_CPP_MIN_LOG_LEVEL는 로그를 담당하는 TensorFlow 환경 변수로,
#1 = INFO, 2 = WARNING, 3 = ERROR(권장하지 않음) 로그를 filter 한다.

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
TRAIN_LOG_DIR = os.path.join('Log/train/', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
TRAIN_CHECK_POINT = 'check_point/'

#Experiment, Optimization options
DATA_SPLIT_PATH = 'data_split.pkl'
BATCH_SIZE = 10
NUM_CLASSES = 11
CROP_SZIE = 112
CHANNEL_NUM = 3
CLIP_LENGTH = 16
EPOCH_NUM = 50
LEARNING_RATE = 1e-4

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## 1) Load UCF11(UCF YouTube Action) dataset Path

In [2]:
DATA_SPLIT_PATH = 'data_split.pkl'
ucf11_dataset = pickle.load(open(DATA_SPLIT_PATH,'rb'))
train_set = ucf11_dataset['train']
test_set = ucf11_dataset['test']

## 2) Get shuffle index

In [3]:
train_video_indices = data_processing.get_video_indices(len(train_set))
test_video_indices = data_processing.get_video_indices(len(test_set))

## 3) Set Default Graph

In [4]:
#with tf.Graph().as_default():
    # Define Graph
batch_clips = tf.placeholder(tf.float32, [BATCH_SIZE, CLIP_LENGTH, CROP_SZIE, CROP_SZIE, CHANNEL_NUM], name='X')
batch_labels = tf.placeholder(tf.int32, [BATCH_SIZE, NUM_CLASSES], name='Y')
keep_prob = tf.placeholder(tf.float32)
logits = C3D_model.C3D(batch_clips, NUM_CLASSES, keep_prob)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_labels))
    tf.summary.scalar('entropy_loss', loss)

with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(batch_labels, 1)), np.float32))
    tf.summary.scalar('accuracy', accuracy)


optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
saver = tf.train.Saver()
summary_op = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### - Set Graph And Train C3D

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    train_summary_writer = tf.summary.FileWriter(TRAIN_LOG_DIR, sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    step = 0
    for epoch in range(EPOCH_NUM):
        accuracy_epoch = 0
        loss_epoch = 0
        batch_index = 0
        for i in range(len(train_video_indices) // BATCH_SIZE):
            step += 1
                # Get Batch for training
            batch_data, batch_index = data_processing.get_batches(train_set, NUM_CLASSES, batch_index,
                                                         train_video_indices, BATCH_SIZE)
                # Train
            _, loss_out, accuracy_out, summary = sess.run([optimizer, loss, accuracy, summary_op],
                                                              feed_dict={batch_clips:batch_data['clips'],
                                                              batch_labels:batch_data['labels'],
                                                                        keep_prob: 0.5})
            loss_epoch += loss_out
            accuracy_epoch += accuracy_out

            if i % 10 == 0:
                print('Epoch %d, Batch %d: Loss is %.5f; Accuracy is %.5f'%(epoch+1, i, loss_out, accuracy_out))
                train_summary_writer.add_summary(summary, step)

        print('Epoch %d: Average loss is: %.5f; Average accuracy is: %.5f'%(epoch+1, loss_epoch / (len(train_video_indices) // BATCH_SIZE),
                                                                                accuracy_epoch / (len(train_video_indices) // BATCH_SIZE)))
        accuracy_epoch = 0
        loss_epoch = 0
        batch_index = 0
        #Get Validation Results
        for i in range(len(test_video_indices) // BATCH_SIZE):
            batch_data, batch_index = data_processing.get_batches(test_set, NUM_CLASSES, batch_index,
                                                                      test_video_indices, BATCH_SIZE)
            loss_out, accuracy_out = sess.run([loss, accuracy],
                                                  feed_dict={batch_clips:batch_data['clips'],
                                                             batch_labels:batch_data['labels'],
                                                            keep_prob: 1.0})
            loss_epoch += loss_out
            accuracy_epoch += accuracy_out

        print('Test loss is %.5f; Accuracy is %.5f'%(loss_epoch / (len(test_video_indices) // BATCH_SIZE),
                                                               accuracy_epoch /(len(test_video_indices) // BATCH_SIZE)))
        saver.save(sess, TRAIN_CHECK_POINT + 'c3d_ckpt', global_step=epoch)

Epoch 1, Batch 0: Loss is 3.72930; Accuracy is 0.10000
Epoch 1, Batch 10: Loss is 2.65427; Accuracy is 0.10000
Epoch 1, Batch 20: Loss is 2.42688; Accuracy is 0.10000
Epoch 1, Batch 30: Loss is 2.43753; Accuracy is 0.00000
Epoch 1, Batch 40: Loss is 2.43815; Accuracy is 0.00000
Epoch 1, Batch 50: Loss is 2.37069; Accuracy is 0.00000


### 